In [ ]:
import gnl.xarray

def get_2d_field(name):
    return xr.open_dataset(f"/home/disk/eos4/nbren12/Data/id/0c31ce4c6763d8ec2abccafe6bde2fa0bed8124d/data/2d/{name}.nc",
                           chunks={'time':100})

def plot_field_2d(name):
    a  = get_2d_field(name)[name].coarsen(x=8).compute()
    plt.figure(figsize=(4,8))
    a.plot()

In [ ]:
%pylab inline

In [ ]:
import xarray as xr
from gnl.xarray import integrate
from thermo import *

import seaborn as sns
sns.set_palette('colorblind')

In [ ]:
q1 = xr.open_dataset("wd/calc/q1.nc").q1
q2 = xr.open_dataset("wd/calc/q2.nc").q2

stats= xr.open_dataset("wd/stat.nc")
rho =stats.RHO[-1]
p = stats.p

In [ ]:
q1p = q1.sel(time=slice(20,None)).mean(['x','time'])
q2p = q2.sel(time=slice(20,None)).mean(['x','time'])

In [ ]:
plt.figure(figsize=(3,5))
plt.plot(q1p.values-1.5, p, label=r'$Q_1$')
plt.plot(-q2p.values, p, label=r'$-Q_2$')
plt.plot(stats.TTEND[-1], p, label=r'$Q_r$')
ylim([1000, 10])
plt.legend(loc="upper right")
plt.ylabel(['p [hPa]'])
plt.xlabel('K/day')
plt.savefig("doc/average_q1q2.png")

In [ ]:
heating_col_int_fac=(Q_(1, "K/day * kg/m^2")* mc.Cp_d).to("W/m^2").magnitude
P1 = integrate(q1*rho, 'z') *heating_col_int_fac
P2 = integrate(q2*rho, 'z') * heating_col_int_fac

In [ ]:
fac=  (Q_(1, "W/m^2") / mc.Lv / Q_(1e3, "kg/m^3")).to("mm/day")
shf = get_2d_field("SHF").SHF.coarsen(x=64)
lhf = get_2d_field("LHF").LHF.coarsen(x=64)

prec = get_2d_field("Prec").Prec.coarsen(x=64)



truth = prec + shf

In [ ]:
def resample_scatter(x,y, **kwargs):
    y = y.reindex_like(x)
    
    return plt.hexbin(x.values.ravel(), y.values.ravel(), **kwargs)

In [ ]:
((Q_(1, "W/m^2")/mc.Lv).to("kg/m^2/s")/Q_(1e3,"kg/m^3")).to("mm/day")

In [ ]:
prec_fac = Q_(1, "mm/day") * Q_(1e3, "kg/m^3") * mc.Lv
prec_wm2 = prec * prec_fac.to("W/m^2").magnitude

P1.mean('x').plot()
((lhf-P2).mean('x')).plot()

prec_wm2.mean('x').plot()
(shf+prec_wm2).mean('x').plot()

xlim([40,50])
ylim([75, 250])
title('Heat Flux and Precip [W/m^2]')
legend(['P1', 'P2', 'Prec','Prec+SHF'])
plt.savefig("doc/q1q2-vs-precip.png")

## 2D plots

In [ ]:
plot_field_2d("U850")
plt.savefig("doc/U850.png")

In [ ]:
plot_field_2d("TB")
plt.savefig("doc/TB.png")